<a href="https://colab.research.google.com/github/Bmiguez/Arezzo-Valuation/blob/main/valuation_arezzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando inicialmente as bibliotecas necessárias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# Fazendo a conexão do collab com o Drive
sys.path.append('/content/drive/MyDrive')

In [ ]:
# Definindo ticker e índice de referencia
ticker = "ARZZ3.SA"
indice = "^BVSP"

# Definindo a data de referência e o intervalo de dados
ano = 2023
mes = 1
dia = 1

data_ref = datetime.date(ano, mes, dia)
data_ini = data_ref - relativedelta(months = 46)

In [ ]:
from ast import increment_lineno
# Extrair retornos ajustados da empresa e do benchmark

retornos = yf.download([ticker, indice], start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = pd.DataFrame(retornos.dropna())

y = retornos[ticker]
x = retornos[indice]



[*********************100%***********************]  2 of 2 completed


In [ ]:
# Começando a regressão

x = sm.add_constant(x)

modelo = sm.OLS(y, x)

resultados = modelo.fit()

print(resultados.summary())
# Extraindo valor do Beta

beta = resultados.params[1]

conf_interval = resultados.conf_int(alpha=0.05)

lower_bound = conf_interval.iloc[1, 0]
upper_bound = conf_interval.iloc[1, 1]

print("Intervalo de Confiança:")
print("Limite Inferior:", lower_bound)
print("Limite Superior:", upper_bound)


                            OLS Regression Results                            
Dep. Variable:               ARZZ3.SA   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     52.80
Date:                Thu, 15 Jun 2023   Prob (F-statistic):           5.30e-09
Time:                        01:25:23   Log-Likelihood:                 55.153
No. Observations:                  45   AIC:                            -106.3
Df Residuals:                      43   BIC:                            -102.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.011      0.978      0.3

In [ ]:
# Cálculo do Custo do equity

# Desalavancando beta

tax = 0.27
debt = 1060093
equity = 2716661

unlev_beta = beta/(1+(1-tax)*debt/equity)

# Risk Free Obtido 14/06
rf = 10.96%

# Default spread 14/06
dfs = 3.14%



# Calculando equity premium

data_ini = data_ref - relativedelta(years = 5)

retornos = yf.download(indice, start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = retornos.dropna()

retorno_anualizado = (float(retornos.mean())+1)**12-1
print(retorno_anualizado)

[*********************100%***********************]  1 of 1 completed
0.08755353683124478


In [3]:
# Montando Valuation
!git clone https://github.com/Bmiguez/Arezzo-Valuation

Cloning into 'Arezzo-Valuation'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), 525.18 KiB | 1.84 MiB/s, done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')